In [1]:
from typing import TypeVar

import sqlalchemy as sa
from sqlalchemy import NullPool

import pandas as pd

In [2]:
SelfClickHouseConnection = TypeVar("SelfClickHouseConnection", bound="ClickHouseConnection")

class ClickHouseConnection:
    """Класс контекстного менеджера для работы с ClickHouse"""
    
    def __init__(self, host: str, username: str, password: str, port: str = "9000"):
        """
        Параметры:
            host: хост ClickHouse
            username: имя пользователя ClickHouse
            password: пароль пользователя ClickHouse
            port: порт ClickHouse
        """
        
        self.session = None
        self.__host = host
        self.__port = port
        self.__username = username
        self.__password = password
        
    def __get_url(self) -> str:
        """Метод формирует connection string для подключения к ClickHouse"""
        
        url = "clickhouse+native://{username}:{password}@{host}:{port}"
        url = url.format(
            host = self.__host,
            port = self.__port,
            username = self.__username,
            password = self.__password
        )
        return url
        
    def __enter__(self: SelfClickHouseConnection) -> SelfClickHouseConnection:
        engine = sa.create_engine(self.__get_url(), poolclass=NullPool)
        self.session = engine.connect()
        return self
    
    def __exit__(self, exception_type, exception_value, traceback):
        self.session.close()
        
    def read_sql(self, query: str) -> pd.DataFrame:
        """
        Запрос к БД через фреймворк pandas
        
        Параметры:
            query: SQL в формате строки
            
        Возвращает:
            датафрейм с результатами запроса
        """
        
        with self as connector:
            return pd.read_sql(query, con=connector.session)

In [3]:
host = "10.120.1.11"
username = "amarbuliev"
password = "KuUNgVlPQiSN6dRqk"

connection_manager = ClickHouseConnection(host=host, username=username, password=password)

In [4]:
df = connection_manager.read_sql("SELECT * FROM default.cabinet_projects LIMIT 1")

In [5]:
df

,_id,project_id,name,head_id,link,department,students_id,direction_head_id
0,6440d0a2de620b321e20c48e,1529,PianoLED: визуализация MIDI сигнала,,https://cabinet.miem.hse.ru/#/project/1529/,ДКИ,,


In [11]:
all_tables = connection_manager.read_sql("SELECT * FROM system.tables")

In [15]:
pd.set_option('display.max_rows', None)
all_tables.head(150)

,database,name,uuid,engine,is_temporary,data_paths,metadata_path,metadata_modification_time,dependencies_database,dependencies_table,...,active_parts,total_marks,lifetime_rows,lifetime_bytes,comment,has_own_data,loading_dependencies_database,loading_dependencies_table,loading_dependent_database,loading_dependent_table
0,INFORMATION_SCHEMA,COLUMNS,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
1,INFORMATION_SCHEMA,KEY_COLUMN_USAGE,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
2,INFORMATION_SCHEMA,REFERENTIAL_CONSTRAINTS,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
3,INFORMATION_SCHEMA,SCHEMATA,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
4,INFORMATION_SCHEMA,TABLES,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
5,INFORMATION_SCHEMA,VIEWS,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
6,INFORMATION_SCHEMA,columns,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
7,INFORMATION_SCHEMA,key_column_usage,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
8,INFORMATION_SCHEMA,referential_constraints,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]
9,INFORMATION_SCHEMA,schemata,00000000-0000-0000-0000-000000000000,View,0,[],,1970-01-01 03:00:00,[],[],...,NaN,NaN,None,None,,0,[],[],[],[]


In [18]:
all_tables['name'].tolist()

['COLUMNS',
 'KEY_COLUMN_USAGE',
 'REFERENTIAL_CONSTRAINTS',
 'SCHEMATA',
 'TABLES',
 'VIEWS',
 'columns',
 'key_column_usage',
 'referential_constraints',
 'schemata',
 'tables',
 'views',
 'cabinet_projects',
 'footprint_users',
 'jitsiusers',
 'nvr_records',
 'wekan_cards',
 'zulip_messages',
 'zulip_streams',
 'COLUMNS',
 'KEY_COLUMN_USAGE',
 'REFERENTIAL_CONSTRAINTS',
 'SCHEMATA',
 'TABLES',
 'VIEWS',
 'columns',
 'key_column_usage',
 'referential_constraints',
 'schemata',
 'tables',
 'views',
 'cabinet_projects',
 'footprint_users',
 'gitlab_groups',
 'gitlab_projects',
 'gitlab_users',
 'jitsiusers',
 'nvr_records',
 'taiga_cards',
 'taiga_people',
 'wekan_cards',
 'zulip_messages',
 'zulip_streams',
 'cabinet_projects',
 'footprint_users',
 'jitsiusers',
 'map_wekan_card2username',
 'nvr_records',
 'stepik_courses',
 'stepik_courses_categories_map',
 'stepik_valid_categories',
 'vedenskiyds_test',
 'wekan_cards',
 'zulip_messages',
 'zulip_streams',
 'aggregate_function_combin